In [1]:
1+1

2

In [2]:
!whoami

desktop-1qpu8cc\¤Ё ­  вагЎЁ­ 


# Заголовок

In [14]:
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('simple_app').master('local[*]').getOrCreate()
df = spark.read.format('csv').option('inferSchema', 'true').option('header', 'true').load('../data/clients.csv')

RuntimeError: Java gateway process exited before sending its port number

In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('../data/transactions.csv')

In [8]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
id,929135.0,464567.000000,268218.315518,0.0,232283.5,464567.0,696850.5,929134.0
client_id,929135.0,500.573223,288.664178,1.0,251.0,501.0,750.0,1000.0
product_category,929135.0,9.348996,7.495062,1.0,5.0,6.0,11.0,29.0
amount,929135.0,1143.750824,3948.343863,50.0,83.0,347.0,1042.0,149891.0


In [9]:
df.head(1)

,id,client_id,product_category,product_company,subtype,amount,date,transaction_type
0,34478,37,29,NaN,Автоплатеж,1209,2020-01-01 00:00:00,Negative


In [12]:
df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])

C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\3059667985.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])


In [13]:
def make_fake_df(df, shift, max_id):

    df_fake = df.copy()
    df_fake.loc[:, 'date'] += pd.offsets.DateOffset(years=shift)
    df_fake.loc[:, 'id'] += df['id'].max() + 1
    df_fake.loc[:, 'amount'] *= np.random.choice(range(11, 20, 1))/10.

    df_fake.loc[:, 'client_id'] = np.where(df_fake['client_id'] < 100, df_fake['client_id'] + np.random.choice(range(1, 10, 1)), df['client_id'])
    df_fake.loc[:, 'client_id'] = np.where(df_fake['client_id'] > 900, df_fake['client_id'] - np.random.choice(range(1, 10, 1)), df['client_id'])
    df_fake.loc[:, 'client_id'] = np.where((df_fake['client_id'] >= 100) & (df_fake['client_id'] < 500) , df_fake['client_id'] + np.random.choice(range(1, 10, 1)), df['client_id'])
    df_fake.loc[:, 'client_id'] = np.where((df_fake['client_id'] >= 500) & (df_fake['client_id'] <= 900) , df_fake['client_id'] - np.random.choice(range(1, 10, 1)), df['client_id'])

    return df_fake

In [14]:
df.head(5)

,id,client_id,product_category,product_company,subtype,amount,date,transaction_type
0,34478,37,29,NaN,Автоплатеж,1209,2020-01-01,Negative
1,65406,71,4,YouTube Music,Подписка,169,2020-01-01,Negative
2,82186,88,29,NaN,Автоплатеж,3711,2020-01-01,Negative
3,83113,89,29,NaN,Автоплатеж,6443,2020-01-01,Negative
4,131373,141,29,NaN,Автоплатеж,7319,2020-01-01,Negative


In [15]:
df_raw = df.copy()

for i in range(1, 13):

    max_id = df['id'].max()
    df_fake = make_fake_df(df_raw, i, max_id)
    df = pd.concat(objs=[df, df_fake])

df.reset_index(drop=True)

C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fake.loc[:, 'date'] += pd.offsets.DateOffset(years=shift)
C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fake.loc[:, 'date'] += pd.offsets.DateOffset(years=shift)
C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace inst

,id,client_id,product_category,product_company,subtype,amount,date,transaction_type
0,34478,37,29,NaN,Автоплатеж,1209.0,2020-01-01 00:00:00,Negative
1,65406,71,4,YouTube Music,Подписка,169.0,2020-01-01 00:00:00,Negative
2,82186,88,29,NaN,Автоплатеж,3711.0,2020-01-01 00:00:00,Negative
3,83113,89,29,NaN,Автоплатеж,6443.0,2020-01-01 00:00:00,Negative
4,131373,141,29,NaN,Автоплатеж,7319.0,2020-01-01 00:00:00,Negative
...,...,...,...,...,...,...,...,...
12078750,940772,13,6,Стрелка,Покупка,55.0,2032-12-31 23:58:04,Negative
12078751,1108473,194,6,NaN,Покупка,59.4,2032-12-31 23:58:11,Negative
12078752,1440379,546,3,Ситимобил,Покупка,344.3,2032-12-31 23:58:13,Negative
12078753,1143928,232,6,NaN,Покупка,95.7,2032-12-31 23:59:03,Negative


In [16]:
df.loc[:, 'date'] -= pd.offsets.DateOffset(years=10)

C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\1613476043.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] -= pd.offsets.DateOffset(years=10)


In [17]:
df_raw = df.copy()

for i in range(1, 13):

    max_id = df['id'].max()
    df_fake = make_fake_df(df_raw, i, max_id)
    df = pd.concat(objs=[df, df_fake])

df.reset_index(drop=True)

C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fake.loc[:, 'date'] += pd.offsets.DateOffset(years=shift)
C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fake.loc[:, 'date'] += pd.offsets.DateOffset(years=shift)
C:\Users\Диана Трубина\AppData\Local\Temp\ipykernel_29768\535486400.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace inst

MemoryError: Unable to allocate 1.17 GiB for an array with shape (1, 157023815) and data type object